In [1]:
# Code reference: https://huggingface.co/docs/transformers/tasks/language_modeling

# Transformers installation
! pip install transformers datasets
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

! pip install transformers[torch]
! pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 6.5 MB/s eta 0:00:00


In [3]:
import os
import pickle

from datasets import Dataset
from transformers import AutoTokenizer

In [16]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
dir_name = "drive/MyDrive/babylm_augment/"

babylm_list_dataset = []
for file_name in os.listdir(dir_name):
    if file_name.endswith(".pkl"):
        with open(dir_name + file_name, "rb") as f:
            print(f"Loading chunk {file_name}")
            data_chunk = pickle.load(f)
            babylm_list_dataset += data_chunk
print(f"Length of babylm_dataset: {len(babylm_list_dataset)}")

Loading chunk babylm_augment_0.pkl
Loading chunk babylm_augment_1.pkl
Loading chunk babylm_augment_2.pkl
Loading chunk babylm_augment_3.pkl
Loading chunk babylm_augment_6.pkl
Loading chunk babylm_augment_4.pkl
Loading chunk babylm_augment_5.pkl
Loading chunk babylm_augment_13.pkl
Loading chunk babylm_augment_18.pkl
Loading chunk babylm_augment_16.pkl
Loading chunk babylm_augment_10.pkl
Loading chunk babylm_augment_14.pkl
Loading chunk babylm_augment_17.pkl
Loading chunk babylm_augment_19.pkl
Loading chunk babylm_augment_12.pkl
Loading chunk babylm_augment_15.pkl
Loading chunk babylm_augment_11.pkl
Loading chunk babylm_augment_30.pkl
Length of babylm_dataset: 15678


In [8]:
babylm_dataset = Dataset.from_list(babylm_list_dataset)
babylm_dataset = babylm_dataset.train_test_split(test_size=0.2)
babylm_dataset

DatasetDict({
    train: Dataset({
        features: ['source', 'text', 'emotion', 'augmented_text'],
        num_rows: 12542
    })
    test: Dataset({
        features: ['source', 'text', 'emotion', 'augmented_text'],
        num_rows: 3136
    })
})

In [9]:
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["augmented_text"]])

block_size = 128

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [10]:
tokenized_babylm = babylm_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=babylm_dataset["train"].column_names,
)

Map (num_proc=4):   0%|          | 0/12542 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1103 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1431 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2229 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2444 > 1024). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/3136 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1110 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1101 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2091 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1512 > 1024). Running this sequence through the model will result in indexing errors


In [11]:
babylm_dataset = tokenized_babylm.map(group_texts, batched=True, num_proc=4)
babylm_dataset

Map (num_proc=4):   0%|          | 0/12542 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3136 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 104272
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 26055
    })
})

In [12]:
from transformers import DataCollatorForLanguageModeling
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

model = AutoModelForCausalLM.from_pretrained("distilgpt2")

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [18]:
training_args = TrainingArguments(
    output_dir="distilgpt2_babylm_clm-model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
    num_train_epochs=3,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=babylm_dataset["train"],
    eval_dataset=babylm_dataset["test"],
    data_collator=data_collator,
)

In [19]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.934300,0.864265
2,0.871700,0.807913


Epoch,Training Loss,Validation Loss
1,0.934300,0.864265
2,0.871700,0.807913
3,0.839400,0.791200


TrainOutput(global_step=39102, training_loss=0.9442790013238613, metrics={'train_runtime': 3402.7587, 'train_samples_per_second': 91.93, 'train_steps_per_second': 11.491, 'total_flos': 1.0217225541648384e+16, 'train_loss': 0.9442790013238613, 'epoch': 3.0})

In [20]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 2.21


In [25]:
trainer.tokenizer = tokenizer

In [26]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/chantomkit/distilgpt2_babylm_clm-model/commit/4d8cb4ccaca01753c6434e0f93e2cd7f6feb2f66', commit_message='End of training', commit_description='', oid='4d8cb4ccaca01753c6434e0f93e2cd7f6feb2f66', pr_url=None, pr_revision=None, pr_num=None)

In [22]:
"""
Testing the finetuned model
"""

prompt = "An apple falls from the tree"

In [33]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("chantomkit/distilgpt2_babylm_clm-model")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer(prompt, return_tensors="pt").input_ids

In [36]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("chantomkit/distilgpt2_babylm_clm-model")
outputs = model.generate(inputs, max_new_tokens=500, do_sample=True, top_k=50, top_p=0.95, pad_token_id=tokenizer.pad_token_id)

In [39]:
tokenizer.batch_decode(outputs, skip_special_tokens=True)

["An apple falls from the tree d   i t.   T h e y   w e r e   c o l l e a g u e s   a n d   d i s a p p o i n t e d.   T h e y   w e r e   a l w a y s   c u r i o u s   a b o u t   t h e   w o r l d   a r o u n d   h e r   a n d   a d m i r a t i o n   t o   m a k e   h e r   s u r e.   I   f e l t   g r a t e f u l   f o r   t h e   a l l   t h a t   s u p p p o r t i c a l   p r o u d   o f   t h e i r   r e f u s t r y,   a n d   m o r e   t h e y   s u p p p p o r t i c a l   d on't   h e l p   m i n d.   T h e y    s u p p p p o r t i c e d   f o r   t o   a    t h e i r   f r o o n g,   a n d   t h e i r   f u s t a t u r a t i o n    b e g a n    o n  t o    t h e i r    e x c l i t t t t t i n u r e s.    H e    s a m   w a i r m e    s t h o o u g h    t o   f o r    s u p p p p    i n    o f f f f    t h e i r    l i f e a d,    s t o r t s e p t i"]